In [ ]:
%cd ../scripts 
from address import addressToCords
import pandas as pd

## Getting Coordinates for listing

Note: There were about 350 listings were the coordinates did not exist on the API due to either spelling mistakes, mismatch of suburb names/street name etc. These were examined and the coordinates were either retrieved manually or discarded. 

The final dataset after manual inspection is contained in listing_w_cords.csv in the same directory as code on the shared google drive

In [ ]:
listing = pd.read_csv("../data/curated/cleaned_property_listing_data.csv")
listing["Coordinates"] = None
listing["Latitude"] = None
listing["Longitude"] = None

In [ ]:
n = 100
start = 0
end = 100

In [ ]:
#Dividing the dataframe into chunks to prevent loss of progress if the Nominatium (API) server goes down
while end < len(listing):
    print(start)
    try: 
        listing["Coordinates"].iloc[start: end] = listing["address"].iloc[start:end].apply(lambda x: addressToCords(x))
        listing["Latitude"].iloc[start:end] = listing["Coordinates"].iloc[start:end].apply(lambda x: x[0])
        listing["Longitude"].iloc[start:end] = listing["Coordinates"].iloc[start:end].apply(lambda x: x[1])
        listing.to_csv("../data/curated/listings_w_cords.csv", index = False)
    except:
        break
    start = end 
    end += n

## Getting Coordinates for Universities

Similarly with Universities, around about half the coordinates were retrieved manually as the API would not return any results. 

In [ ]:
universities = pd.read_csv("../data/raw/List of Universities.csv")

In [ ]:
# Using the api to search for the coordinates of the University
universities.loc[universities['Name'].isnull(), 'Name'] = ""
universities["Coordinates"] = (universities["University"] + " " + universities["Suburb or town"] + " " + universities["Name"]).apply(lambda x: addressToCords(x))
universities["Latitude"] = universities["Coordinates"].apply(lambda x: x[0])
universities["Longitude"] = universities["Coordinates"].apply(lambda x: x[1])

In [ ]:
universities[["University", "Suburb or town", "Name", "Latitude", "Longitude"]].to_csv("../data/curated/universities.csv", index=False)